## Cloning Facebook Research TransCoder Repository

In [1]:
!git clone https://github.com/facebookresearch/TransCoder.git

Cloning into 'TransCoder'...
remote: Enumerating objects: 2194, done.
remote: Counting objects: 100% (2194/2194), done.
remote: Compressing objects: 100% (1106/1106), done.
remote: Total 2194 (delta 1088), reused 2191 (delta 1085), pack-reused 0
Receiving objects: 100% (2194/2194), 4.03 MiB | 15.81 MiB/s, done.
Resolving deltas: 100% (1088/1088), done.


## Installing FastBPE

In [2]:
!git clone https://github.com/glample/fastBPE
%cd fastBPE
!g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast
!python setup.py install
%cd ..

Cloning into 'fastBPE'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 59 (delta 0), reused 2 (delta 0), pack-reused 54
Unpacking objects: 100% (59/59), done.
/content/fastBPE
Compiling fastBPE/fastBPE.pyx because it changed.
[1/1] Cythonizing fastBPE/fastBPE.pyx
running install
running bdist_egg
running egg_info
creating fastBPE.egg-info
writing fastBPE.egg-info/PKG-INFO
writing dependency_links to fastBPE.egg-info/dependency_links.txt
writing top-level names to fastBPE.egg-info/top_level.txt
writing manifest file 'fastBPE.egg-info/SOURCES.txt'
package init file 'fastBPE/__init__.py' not found (or not a regular file)
reading manifest template 'MANIFEST.in'
writing manifest file 'fastBPE.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
building 'fastBPE' extension
creating build
creating build/temp.linux-x8

## Installing PyTorch

Checking CUDA version to install the compatible PyTorch version

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


Install the correct PyTorch version

In [4]:
%pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


##Installing SacreBleu

In [5]:
%pip install sacrebleu=="1.2.11"

     |████████████████████████████████| 81kB 5.1MB/s 
  Created wheel for sacrebleu: filename=sacrebleu-1.2.11-cp36-none-any.whl size=18641 sha256=fab6129dcfbffa8e1fa169c2c0069cbb2071be3df7bd20dec73775063201924b
  Stored in directory: /root/.cache/pip/wheels/93/0f/06/e1c5dcbca58e907c17b59be8e1f10ae4e43bb1fb68197a8d7c
  Created wheel for typing: filename=typing-3.7.4.3-cp36-none-any.whl size=26309 sha256=1f753426f146b7959b9a2d1aa6f8d1438c2794ba348d7ad877fff832e46c81d0
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
Successfully built sacrebleu typing


##Installing Clang

In [6]:
%pip install clang

Update the code to point to the correct llvm path

In [7]:
%cd /usr/lib/llvm-6.0/lib
!ln -s libclang.so.1 libclang.so
%cd /content
!sed -i "s|clang.cindex.Config.set_library_path('/usr/lib/llvm-7/lib/')|clang.cindex.Config.set_library_path('/usr/lib/llvm-6.0/lib/')|g" /content/TransCoder/preprocessing/src/code_tokenizer.py

/usr/lib/llvm-6.0/lib
/content


## Downloading the model checkpoints

First checkpoint for C++ -> Java, Java -> C++ and Java -> Python


In [8]:
%mkdir checkpoints
%cd checkpoints
!wget https://dl.fbaipublicfiles.com/transcoder/model_1.pth

/content/checkpoints
--2020-08-09 22:17:45--  https://dl.fbaipublicfiles.com/transcoder/model_1.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622089114 (593M) [application/octet-stream]
Saving to: ‘model_1.pth’

model_1.pth         100%[===================>] 593.27M  66.4MB/s    in 9.3s    

2020-08-09 22:17:55 (63.6 MB/s) - ‘model_1.pth’ saved [622089114/622089114]



Second checkpoint for C++ -> Python, Python -> C++ and Python -> Java


In [9]:
!wget https://dl.fbaipublicfiles.com/transcoder/model_2.pth
%cd ..

--2020-08-09 22:17:56--  https://dl.fbaipublicfiles.com/transcoder/model_2.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622089114 (593M) [application/octet-stream]
Saving to: ‘model_2.pth’

model_2.pth         100%[===================>] 593.27M  50.6MB/s    in 11s     

2020-08-09 22:18:07 (55.9 MB/s) - ‘model_2.pth’ saved [622089114/622089114]

/content


##Running the model for translation

Update the code to point to the correct BPE codes file path

In [10]:
!sed -i 's|default="data/BPE_with_comments_codes"|default="/content/TransCoder/data/BPE_with_comments_codes"|g' /content/TransCoder/translate.py

Finally, we reached the interesting part. Let's translate some code files.

First, upload the files you want to translate into another language among (cpp, java and python)

In [15]:
%mkdir -p examples
%cd /content/examples
from google.colab import files
files.upload()
%cd /content

/content/examples


Saving 2.cpp to 2.cpp
/content


Second, update the following command to reflect your chosen src lang, tgt lang, the model checkpoint path as mentioned in the previous section and the uploaded input file path. 

In [16]:
INPUT_FILE_PATH = '/content/examples/2.cpp'
SRC_LANG = 'cpp'
TGT_LANG = 'python'
MODEL_PATH = '/content/checkpoints/model_1.pth' if (SRC_LANG, TGT_LANG) in [('cpp', 'java'), ('java', 'cpp'), ('java', 'python')] else '/content/checkpoints/model_2.pth'

!cat $INPUT_FILE_PATH
print('=' * 20)
!echo 'Using checkpoint: $MODEL_PATH'
!python /content/TransCoder/translate.py --src_lang $SRC_LANG --tgt_lang $TGT_LANG --model_path $MODEL_PATH < $INPUT_FILE_PATH

#include <bits/stdc++.h>

using namespace std;

int main()
{
    const int MAX_SIZES = 3;
    map<string,int> size_map = {{"small",0}, {"medium", 1}, {"large", 2}};
    int T, C, N;
    cin >> T;
    assert(T > 0);
    while(T--){
        cin >> C >> N;
        assert(C > 0 && N > 0);
        map<string, vector<long long> > coffee;
        for(int i = 0; i < C; i++){
            string coffee_name;
            cin >> coffee_name;
            long long price;
            for(int j = 0; j < MAX_SIZES; j++){
                cin >> price;
                coffee[coffee_name].push_back(price);
            }
        }
        int delivery_cost = 100/N;
        for(int i = 0; i < N; i++){
            string person_name, coffee_size, coffee_name;
            cin >> person_name >> coffee_size >> coffee_name;
            long long total_cost = coffee[coffee_name][size_map[coffee_size]] + delivery_cost;
            if(total_cost % 5 == 1) total_cost--;
            else if(total_cost % 5 == 4) tota